# B tagging classification with a Transformer model

**Author:** Ethan Jiang based on [Theodoros Ntakouris](https://github.com/ntakouris)<br>

## Introduction

This is the Transformer architecture from
[Attention Is All You Need](https://arxiv.org/abs/1706.03762),
applied to timeseries instead of natural language.

This example requires TensorFlow 2.9.1.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

2023-03-27 21:37:56.340237: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-27 21:37:56.340261: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
file_str = 'Jan06_FlavFix_smear_1_std_xtd_zst.h5'

In [3]:
f5 = h5py.File(file_str, 'r')
x_train = np.array( f5['x_train'] )
y_train = to_categorical( np.array( f5['y_train'] ) )

n_classes = len(np.unique(y_train, axis=0))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

x_test = np.array( f5['x_test'] )
y_test = to_categorical ( np.array( f5['y_test'] ) )

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [4]:
n_classes

3

In [5]:
y_train

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [6]:
x_train.shape

(701516, 15, 6)

## Build the model

Our model processes a tensor of shape `(batch size, sequence length, features)`,
where `sequence length` is the number of time steps and `features` is each input
timeseries.

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

We include residual connections, layer normalization, and dropout.
The resulting layer can be stacked multiple times.

In [8]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    res = layers.Add()([x, inputs])

    # Feed Forward Part
    x = layers.Dense(ff_dim, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)
    x = layers.Add()([x, res])
    return x


In [9]:

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0.15,
    mlp_dropout=0.15,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.Flatten()(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


## Train and evaluate

In [10]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=32,
    num_heads=3,
    ff_dim=8,
    num_transformer_blocks=1,
    mlp_units=[32,16,8],
    mlp_dropout=0.15,
    dropout=0.15,
)

# model.compile(
#     loss="sparse_categorical_crossentropy",
#     optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#     metrics=["sparse_categorical_accuracy"],
# )

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15, 6)]      0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 15, 6)       2598        ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 15, 6)        0           ['multi_head_attention[0][0]']   
                                                                                                  
 add (Add)                      (None, 15, 6)        0           ['dropout[0][0]',            

2023-03-16 08:45:38.441336: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-16 08:45:38.441702: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-16 08:45:38.441735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-16 08:45:38.441763: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-16 08:45:38.441789: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [11]:
print(input_shape)

(15, 6)


In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [13]:
callbacks = [EarlyStopping(verbose=True, patience=10, monitor='val_accuracy')]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=2,
    batch_size=2**14,
    callbacks=callbacks,
)


Epoch 1/2
35/35 [==============================] - 17s 461ms/step - loss: 1.0386 - accuracy: 0.4782 - val_loss: 0.9709 - val_accuracy: 0.5665
Epoch 2/2
35/35 [==============================] - 16s 459ms/step - loss: 0.9628 - accuracy: 0.5534 - val_loss: 0.9087 - val_accuracy: 0.6092


In [14]:
model.evaluate(x_test, y_test, verbose=1)

9396/9396 [==============================] - 8s 867us/step - loss: 0.9074 - accuracy: 0.6090


[0.9073992967605591, 0.6089805364608765]

In [15]:
model.save("b_tag_no_norm_2.h5")

In [12]:
from tensorflow.keras.models import load_model
model = load_model('b_tag_no_norm_1.h5')

2023-03-27 21:38:10.347659: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-27 21:38:10.348053: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-27 21:38:10.348105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-27 21:38:10.348140: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-27 21:38:10.348170: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [14]:
# model.evaluate(x_test, y_test, verbose=1)

In [15]:
import hls4ml
import os
os.environ['PATH'] = '/opt/Xilinx/Vivado/2019.2/bin:' + os.environ['PATH']

In [16]:
config = hls4ml.utils.config_from_keras_model(model, granularity='model', default_precision='ap_fixed<22,8>')
config['Model']['ReuseFactor']=1
config['Model']['Strategy']='Resource'
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir='mha_time_series/hls4ml_prj_reuse1_v2',
                                                       part='xcvu13p-fhga2104-2L-e')

Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: multi_head_attention, layer type: MultiHeadAttention
Layer name: add, layer type: Add
Layer name: dense, layer type: Dense
  -> Activation (relu), layer name: dense
Layer name: dense_1, layer type: Dense
  -> Activation (linear), layer name: dense_1
Layer name: add_1, layer type: Add
Layer name: multi_head_attention_1, layer type: MultiHeadAttention
Layer name: add_2, layer type: Add
Layer name: dense_2, layer type: Dense
  -> Activation (relu), layer name: dense_2
Layer name: dense_3, layer type: Dense
  -> Activation (linear), layer name: dense_3
Layer name: add_3, layer type: Add
Layer name: dense_4, layer type: Dense
  -> Activation (relu), layer name: dense_4
Layer name: dense_5, layer type: Dense
  -> Activation (relu), layer name: dense_5
Layer name: dense_6, layer type: Dense
  -> Activation (relu), layer name: dense_6
Layer name: dense_7, layer type: Dense
  -> Activation (softmax), layer name: den

In [17]:
hls_model.compile()

Writing HLS project
Done


In [18]:
x_test1=np.ascontiguousarray(x_test[5:15,:,:])

In [19]:
# layer_name = 'multi_head_attention'
# intermediate_layer_model = keras.Model(inputs=model.input,
#                                        outputs=model.get_layer(layer_name).output)
# intermediate_output = intermediate_layer_model(x_test1)
# print(intermediate_output[0].numpy().round(5))

In [20]:
y_keras = model.predict(x_test[5:15,:,:]) 
print("y_keras prediction is:")
print(y_keras)

1/1 [==============================] - 0s 178ms/step
y_keras prediction is:
[[0.2102902  0.21076627 0.57894355]
 [0.06808326 0.13465342 0.79726326]
 [0.11175717 0.17712659 0.7111162 ]
 [0.01817898 0.07823452 0.9035865 ]
 [0.8785012  0.07316413 0.04833474]
 [0.28782716 0.1994664  0.51270646]
 [0.7020107  0.1488705  0.14911866]
 [0.06838343 0.14187093 0.7897457 ]
 [0.5710467  0.19556023 0.23339303]
 [0.76355004 0.12496373 0.11148623]]


In [21]:
y_hls = hls_model.predict(x_test1)
print("hls prediction is:")
print(y_hls)

hls prediction is:
[[0.10443115 0.18139648 0.71765137]
 [0.05908203 0.12530518 0.81463623]
 [0.28546143 0.21295166 0.50506592]
 [0.01837158 0.05682373 0.9105835 ]
 [0.1940918  0.17962646 0.62713623]
 [0.04364014 0.08221436 0.87103271]
 [0.75457764 0.13092041 0.1192627 ]
 [0.04699707 0.12011719 0.83081055]
 [0.95593262 0.03057861 0.00524902]
 [0.58111572 0.16973877 0.2489624 ]]


In [22]:
hls_model.build(csim=False)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'ej' on host 'ej-ubt' (Linux_x86_64 version 5.4.0-144-generic) on Mon Mar 27 21:38:23 PDT 2023
INFO: [HLS 200-10] On os Ubuntu 18.04.6 LTS
INFO: [HLS 200-10] In directory '/home/ej/workspace/B_tagging_transformer/mha_time_series/hls4ml_prj_reuse1_v2'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Opening project '/home/ej/workspace/B_tagging_transformer/mha_time_series/hls4ml_prj_reuse1_v2/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the project
INFO: [HLS 200-10] Adding test bench file 'myproject_test.c

INFO: [HLS 200-111] Finished Standard Transforms Time (s): cpu = 00:00:28 ; elapsed = 00:00:29 . Memory (MB): peak = 979.355 ; gain = 499.035 ; free physical = 2010 ; free virtual = 16403
INFO: [HLS 200-10] Checking synthesizability ...
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2019.2/continuous/2019_11_06_2708876/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59

INFO: [HLS 200-111] Finished Checking Synthesizability Time (s): cpu = 00:00:39 ; elapsed = 00:00:40 . Memory (MB): peak = 979.355 ; gain = 499.035 ; free physical = 1992 ; free virtual = 16388
INFO: [XFORM 203-1101] Packing variable 'v_proj.V.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:293) into a 704-bit variable.
INFO: [XFORM 203-1101] Packing variable 'datav_pack.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:129) into a 704-bit variable.
INFO: [XFORM 203-1101] Packing variable 'q_proj.V.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:291) into a 704-bit variable.
INFO: [XFORM 203-1101] Packing variable 'k_proj.V.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:292) into a 704-bit variable.
INFO: [XFORM 203-1101] Packing variable 'datak_pack.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:75) into a 704-bit variable.
INFO: [XFORM 203-1101] Packing variable 'dataq_pack.data.V' (firmware/nnet_utils/nnet_multiheadattention.h:75) into a 704-bit va

INFO: [HLS 200-489] Unrolling loop 'qk' (firmware/nnet_utils/nnet_multiheadattention.h:153) in function 'nnet::matrixmul<ap_fixed<22, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<22, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config9>' completely with a factor of 15.
INFO: [HLS 200-489] Unrolling loop 'col' (firmware/nnet_utils/nnet_multiheadattention.h:157) in function 'nnet::matrixmul<ap_fixed<22, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<22, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config9>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'product' (firmware/nnet_utils/nnet_multiheadattention.h:160) in function 'nnet::matrixmul<ap_fixed<22, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<22, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, config9>' completely with a factor of 15.
INFO: [HLS 200-489] Unrolling loop 'Loop-1.1' (firmware/nnet_utils/nnet_multiheadattention.h:39) in function 'nnet::matrixmul_transpose<ap_fixed<22, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<22, 8, (ap_q_mode)5, (ap

INFO: [HLS 200-489] Unrolling loop 'Loop-1.1' (firmware/nnet_utils/nnet_dense_seq.h:36) in function 'myproject' completely with a factor of 6.
INFO: [HLS 200-489] Unrolling loop 'Loop-2.1' (firmware/nnet_utils/nnet_dense_seq.h:36) in function 'myproject' completely with a factor of 8.
INFO: [HLS 200-489] Unrolling loop 'Loop-3.1' (firmware/nnet_utils/nnet_dense_seq.h:36) in function 'myproject' completely with a factor of 6.
INFO: [HLS 200-489] Unrolling loop 'Loop-4.1' (firmware/nnet_utils/nnet_dense_seq.h:36) in function 'myproject' completely with a factor of 8.
INFO: [HLS 200-489] Unrolling loop 'Loop-1' (firmware/nnet_utils/nnet_activation.h:369) in function 'nnet::softmax_legacy<ap_fixed<22, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<22, 8, (ap_q_mode)5, (ap_o_mode)3, 0>, softmax_config24>' completely with a factor of 3.
INFO: [HLS 200-489] Unrolling loop 'Loop-2' (firmware/nnet_utils/nnet_activation.h:389) in function 'nnet::softmax_legacy<ap_fixed<22, 8, (ap_q_mode)5, (ap_o_mo

...

INFO: [RTMG 210-285] Implementing FIFO 'layer13_out_46_V_c_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer13_out_47_V_c_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer13_out_48_V_c_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer13_out_49_V_c_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer13_out_50_V_c_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer13_out_51_V_c_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer13_out_52_V_c_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer13_out_53_V_c_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer13_out_54_V_c_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer13_out_55_V_c_U(fifo_w22_d2_

INFO: [RTMG 210-285] Implementing FIFO 'layer15_out_38_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer15_out_39_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer15_out_40_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer15_out_41_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer15_out_42_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer15_out_43_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer15_out_44_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer15_out_45_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer15_out_46_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer15_out_47_V_U(fifo_w22_d2_A_x0)' using Shift R

INFO: [RTMG 210-285] Implementing FIFO 'layer17_out_30_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer17_out_31_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer18_out_0_V_U(fifo_w21_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer18_out_1_V_U(fifo_w21_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer18_out_2_V_U(fifo_w21_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer18_out_3_V_U(fifo_w21_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer18_out_4_V_U(fifo_w21_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer18_out_5_V_U(fifo_w21_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer18_out_6_V_U(fifo_w21_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer18_out_7_V_U(fifo_w21_d2_A)' using Shift Registers.
INFO: [RTMG 210-285] I

INFO: [RTMG 210-285] Implementing FIFO 'layer23_out_2_V_U(fifo_w22_d2_A_x0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'start_for_Loop_1_proc_U0_U(start_for_Loop_1_proc_U0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'start_for_relu_ap_fixed_ap_fixed_22_8_5_3_0_relu_config5_U0_U(start_for_relu_ap_fixed_ap_fixed_22_8_5_3_0_relu_config5_U0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'start_for_add_ap_fixed_ap_fixed_ap_fixed_22_8_5_3_0_confi3i2_U(start_for_add_ap_fixed_ap_fixed_ap_fixed_22_8_5_3_0_confi3i2)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'start_for_multiheadattention_ap_fixed_ap_fixed_config9_U0_U(start_for_multiheadattention_ap_fixed_ap_fixed_config9_U0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'start_for_Loop_3_proc_U0_U(start_for_Loop_3_proc_U0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'start_for_relu_ap_fixed_ap_fixed_22_8_5_3_0_relu_config124jc_U(star

{'CSynthesisReport': {'EstimatedClockPeriod': '7.556',
  'BestLatency': '267',
  'WorstLatency': '267',
  'IntervalMin': '56',
  'IntervalMax': '56',
  'BRAM_18K': '1604',
  'DSP48E': '23125',
  'FF': '673598',
  'LUT': '1343382',
  'URAM': '0',
  'AvailableBRAM_18K': '5376',
  'AvailableDSP48E': '12288',
  'AvailableFF': '3456000',
  'AvailableLUT': '1728000',
  'AvailableURAM': '1280'}}

In [23]:
hls4ml.report.read_vivado_report('mha_time_series/hls4ml_prj_reuse1_v2')

Found 1 solution(s) in mha_time_series/hls4ml_prj_reuse1_v2/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject'
* Date:           Tue Mar 28 09:50:56 2023

* Version:        2019.2 (Build 2704478 on Wed Nov 06 22:10:23 MST 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: virtexuplus
* Target device:  xcvu13p-fhga2104-2L-e


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+---------+----------+------------+
    |  Clock |  Target | Estimated| Uncertainty|
    +--------+---------+----------+------------+
    |ap_clk  | 5.00 ns | 7.556 ns |   0.62 ns  |
    +--------+---------+----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+----------+----------+-----+-----+----------+
    |  Latency (cycles) |  Latency (absolute) |  Interval | Pipeline |
    |   min   |   max   |    min   |    max   | min | max |   Type   |
    +---------+-